In [33]:
from pynwb import NWBHDF5IO
from hdmf_zarr.nwb import NWBZarrIO

with NWBHDF5IO('/Users/mavaylon/Research/NWB/hdmf-zarr/docs/gallery/resources/sub_anm00239123_ses_20170627T093549_ecephys_and_ogen.nwb', 'r', load_namespaces=False) as read_io:
    file = read_io.read()

In [34]:
file

root pynwb.file.NWBFile at 0x4866203296
Fields:
  devices: {
    ADunit <class 'pynwb.device.Device'>,
    laser <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    ADunit_32 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: N/A
  experimenter: ['Zengcai Guo']
  file_create_date: [datetime.datetime(2019, 10, 7, 15, 10, 30, 595741, tzinfo=tzoffset(None, -18000))]
  identifier: anm00239123_2017-06-27_09-35-49
  institution: Janelia Research Campus
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  ogen_sites: {
    left-ALM <class 'pynwb.ogen.OptogeneticStimulusSite'>
  }
  related_publications: ['doi:10.1038/nature22324']
  session_description: Extracellular ephys recording of mouse doing discrimination task(lick left/right), with optogenetic stimulation plus pole and auditory stimulus
  session_start_time: 2017-06-27 09:35:49-05:00
  subject: subject pynwb.file.Subject at 0x4866200656
Fields:
  genotype: Ai32 x PV-Cre
  sex: M
  species: Mus musculus
  subject_id: anm00239123

  timestamps_reference_time: 2017-06-27 09:35:49-05:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>
  units: units <class 'pynwb.misc.Units'>

In [32]:
file.electrodes


electrodes hdmf.common.table.DynamicTable at 0x4452016064
Fields:
  colnames: ['x' 'y' 'z' 'imp' 'location' 'filtering' 'group' 'group_name']
  columns: (
    x <class 'hdmf.common.table.VectorData'>,
    y <class 'hdmf.common.table.VectorData'>,
    z <class 'hdmf.common.table.VectorData'>,
    imp <class 'hdmf.common.table.VectorData'>,
    location <class 'hdmf.common.table.VectorData'>,
    filtering <class 'hdmf.common.table.VectorData'>,
    group <class 'hdmf.common.table.VectorData'>,
    group_name <class 'hdmf.common.table.VectorData'>
  )
  description: metadata about extracellular electrodes

In [28]:
file.acquisition

{}

In [31]:
file.electrodes.group.data[0]

ValueError: Invalid dataset identifier (invalid dataset identifier)

In [8]:
from datetime import datetime
from uuid import uuid4

import numpy as np
from dateutil.tz import tzlocal

from pynwb import NWBHDF5IO, NWBFile
from pynwb.ecephys import LFP, ElectricalSeries

nwbfile = NWBFile(
    session_description="my first synthetic recording",
    identifier=str(uuid4()),
    session_start_time=datetime.now(tzlocal()),
    experimenter=[
        "Baggins, Bilbo",
    ],
    lab="Bag End Laboratory",
    institution="University of Middle Earth at the Shire",
    experiment_description="I went on an adventure to reclaim vast treasures.",
    session_id="LONELYMTN001",
)

device = nwbfile.create_device(
    name="array", description="the best array", manufacturer="Probe Company 9000"
)

nwbfile.add_electrode_column(name="label", description="label of electrode")

nshanks = 4
nchannels_per_shank = 3
electrode_counter = 0

for ishank in range(nshanks):
    # create an electrode group for this shank
    electrode_group = nwbfile.create_electrode_group(
        name="shank{}".format(ishank),
        description="electrode group for shank {}".format(ishank),
        device=device,
        location="brain area",
    )
    # add electrodes to the electrode table
    for ielec in range(nchannels_per_shank):
        nwbfile.add_electrode(
            group=electrode_group,
            label="shank{}elec{}".format(ishank, ielec),
            location="brain area",
        )
        electrode_counter += 1

all_table_region = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description="all electrodes",
)

raw_data = np.random.randn(50, 12)
raw_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=raw_data,
    electrodes=all_table_region,
    starting_time=0.0,  # timestamp of the first sample in seconds relative to the session start time
    rate=20000.0,  # in Hz
)

nwbfile.add_acquisition(raw_electrical_series)

with NWBHDF5IO("ecephys_tutorial.nwb", "w") as io:
    io.write(nwbfile)
    
with NWBHDF5IO("ecephys_tutorial.nwb", "r") as io:
    read_nwbfile = io.read()
    
    read_nwbfile.electrodes.group.data[0]
#     print(read_nwbfile.acquisition["ElectricalSeries"])
#     print(read_nwbfile.processing["ecephys"])
#     print(read_nwbfile.processing["ecephys"]["LFP"])
#     print(read_nwbfile.processing["ecephys"]["LFP"]["ElectricalSeries"])   

In [11]:
 
with NWBHDF5IO("ecephys_tutorial.nwb", "r") as io:
    read_nwbfile = io.read()
    
    r=read_nwbfile.electrodes.group.data
r

In [6]:
read_nwbfile

root pynwb.file.NWBFile at 0x4924479792
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    array <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    shank0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    shank1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    shank2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    shank3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: I went on an adventure to reclaim vast treasures.
  experimenter: ['Baggins, Bilbo']
  file_create_date: [datetime.datetime(2023, 9, 13, 10, 8, 12, 39496, tzinfo=tzoffset(None, -25200))]
  identifier: 3ca23677-ee6a-4c43-99dd-5da15c6d833f
  institution: University of Middle Earth at the Shire
  lab: Bag End Laboratory
  session_description: my first synthetic recording
  session_id: LONELYMTN001
  session_start_time: 2023-09-13 10:08:12.034293-07:00
  timestamps_reference_time: 2023-09-13 10:08:12.034293-07:00

In [3]:
read_nwbfile.electrodes.group.data

In [7]:
read_nwbfile.electrodes.to_dataframe()

ValueError: Dset_id is not a dataset id (dset_id is not a dataset ID)

In [1]:
# sphinx_gallery_thumbnail_path = 'figures/gallery_thumbnail_plot_convert_nwb.png'
import os
import shutil

# Input file to convert
basedir = "resources"
filename = os.path.join(basedir, "sub_anm00239123_ses_20170627T093549_ecephys_and_ogen.nwb")
# Zarr file to generate for converting from HDF5 to Zarr
zarr_filename = "test_zarr_" + os.path.basename(filename) + ".zarr"
# HDF5 file to generate for converting from Zarr to HDF5
hdf_filename = "test_hdf5_" + os.path.basename(filename)

# Delete our converted HDF5 and Zarr file from previous runs of this notebook
for fname in [zarr_filename, hdf_filename]:
    if os.path.exists(fname):
        print("Removing %s" % fname)
        if os.path.isfile(fname):  # Remove a single file (here the HDF5 file)
            os.remove(fname)
        else:  # remove whole directory and subtree (here the Zarr file)
            shutil.rmtree(zarr_filename)

###############################################################################
# Convert the NWB file from HDF5 to Zarr
# --------------------------------------
#
# To convert files between storage backends, we use HMDF's :hdmf-docs:`export <export.html>` functionality.
# As this is an NWB file, we here use the :py:class:`pynwb.NWBHDF5IO` backend for reading the file from
# from HDF5 and use the :py:class:`~hdmf_zarr.nwb.NWBZarrIO` backend to export the file to Zarr.

from pynwb import NWBHDF5IO
from hdmf_zarr.nwb import NWBZarrIO

# with NWBHDF5IO(filename, 'r', load_namespaces=False) as read_io:
#     file = read_io.read()
#
with NWBHDF5IO(filename, 'r', load_namespaces=False) as read_io:  # Create HDF5 IO object for read
    with NWBZarrIO(zarr_filename, mode='w') as export_io:         # Create Zarr IO object for write
        export_io.export(src_io=read_io, write_args=dict(link_data=False))   # Export from HDF5 to Zarr

with NWBZarrIO(path=zarr_filename, mode="r") as io:
    infile = io.read()


In [2]:
infile

root pynwb.file.NWBFile at 0x4854283616
Fields:
  devices: {
    ADunit <class 'pynwb.device.Device'>,
    laser <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    ADunit_32 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: N/A
  experimenter: ['Zengcai Guo']
  file_create_date: [datetime.datetime(2019, 10, 7, 15, 10, 30, 595741, tzinfo=tzoffset(None, -18000))]
  identifier: anm00239123_2017-06-27_09-35-49
  institution: Janelia Research Campus
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  keywords: <zarr.core.Array '/general/keywords' (6,) object read-only>
  ogen_sites: {
    left-ALM <class 'pynwb.ogen.OptogeneticStimulusSite'>
  }
  related_publications: ['doi:10.1038/nature22324']
  session_description: Extracellular ephys recording of mouse doing discrimination task(lick left/right), with optogenetic stimulation plus pole and auditory stimulus
  session_start_time: 2017-06-27 09:35:49-05:00
  subject: subject pynwb.file.Subject at 0x4854278624
Fields:
  genotype: Ai32 x PV-Cre
  sex: M
  species: Mus musculus
  subject_id: anm00239123

  timestamps_reference_time: 2017-06-27 09:35:49-05:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>
  units: units <class 'pynwb.misc.Units'>

In [8]:
infile.electrodes.group.data

array([{'source': '.', 'path': '/general/extracellular_ephys/ADunit_32', 0: root/general/extracellular_ephys/ADunit_32 GroupBuilder {'attributes': {'description': 'N/A', 'location': 'brain_region: Thalamus; brain_subregion: N/A; cortical_layer: N/A; hemisphere: left; coordinate_ref: bregma; coordinate_ap: 2.50; coordinate_ml: -1.50; coordinate_dv: 0.00', 'namespace': 'core', 'neurodata_type': 'ElectrodeGroup', 'object_id': 'c9e8c070-688a-4347-9629-539a755a47c7'}, 'groups': {}, 'datasets': {}, 'links': {'device': root/general/extracellular_ephys/ADunit_32/device LinkBuilder {'builder': root/general/devices/ADunit GroupBuilder {'attributes': {'namespace': 'core', 'neurodata_type': 'Device', 'object_id': '6065edb9-03a4-40e1-84f2-1bdc14799ede'}, 'groups': {}, 'datasets': {}, 'links': {}}}}}},
       {'source': '.', 'path': '/general/extracellular_ephys/ADunit_32', 1: root/general/extracellular_ephys/ADunit_32 GroupBuilder {'attributes': {'description': 'N/A', 'location': 'brain_region: Tha